### Notebook to merge machine read triples after extraction and processing from INDRA/REACH and SemRep

In [1]:
import pandas as pd
import numpy as np
import pickle
from rdflib.namespace import OWL, RDF, RDFS
import os
import pickle, json

#Create networkx graph from triples
import glob
import hashlib
import json
import networkx as nx  # type: ignore
import os
import os.path

from collections import Counter  # type: ignore
from more_itertools import unique_everseen  # type: ignore
from rdflib import BNode, Graph, Literal, Namespace, URIRef  # type: ignore
from rdflib.namespace import OWL, RDF, RDFS  # type: ignore
from rdflib.plugins.serializers.nt import _quoteLiteral  # type: ignore
import subprocess

from tqdm import tqdm  # type: ignore
from typing import Dict, List, Optional, Set, Tuple, Union

In [4]:
##read in machine reading graphs
g1 = '../literature_graphs/output_graphs/machineread_semrep_version1.nt'
g2 = '../literature_graphs/output_graphs/machineread_reach_version1.nt'

In [2]:
#if merging ntriples - concatenate files
#if merging gpickle - use nx.compose function. Also merge nodelabels and fix labels

In [5]:
with open(g1,'r') as file1:
    g = file1.read()
graph1 = g.split('\n')
type(graph1)

list

In [6]:
graph1[0]

'<http://purl.obolibrary.org/obo/CHEBI_28748> <http://purl.obolibrary.org/obo/RO_0002213> <http://purl.obolibrary.org/obo/CHEBI_9948> .'

In [7]:
with open(g2,'r') as file2:
    g = file2.read()
graph2 = g.split('\n')
graph2[0]

'<http://purl.obolibrary.org/obo/CHEBI_16057> <http://www.w3.org/2000/01/rdf-schema#label> "Dimethylphenylpiperazinium_Iodide" .'

In [8]:
print(len(graph1), len(graph2))

38141 47287


In [9]:
output_file = '../literature_graphs/output_graphs/machine_read_merged_v2.0.0.nt'

In [10]:
with open(output_file, 'w') as fileo:
    for gt1 in graph1:
        fileo.write(gt1)
        fileo.write('\n')
    for gt2 in graph2:
        fileo.write(gt2)
        fileo.write('\n')
file1.close()
file2.close()
fileo.close()

In [11]:
with open(output_file,'r') as filei:
    g = filei.read()
mr = g.split('\n')
len(mr)

85429

In [13]:
inferred_file = '../literature_graphs/closure_output/inferred-transitive-and-symmetric.ntriples'
with open(inferred_file,'r') as file2:
    g2 = file2.read()
mrinf = g2.split('\n')
len(mrinf)

21633

In [14]:
mr[0]

'<http://purl.obolibrary.org/obo/CHEBI_28748> <http://purl.obolibrary.org/obo/RO_0002213> <http://purl.obolibrary.org/obo/CHEBI_9948> .'

In [15]:
mrinf_new = []
for item in mrinf:
    if mrinf != '':
        temp = item.replace('>', '> ')
        mrinf_new.append(temp)
len(mrinf_new)

21633

In [16]:
outmerged = '../literature_graphs/output_graphs/machine_read_merged_with_closure_v2.0.0.nt'
with open(outmerged, 'w') as fileo:
    for gt1 in mr:
        if gt1 != '':
            fileo.write(gt1)
            fileo.write('\n')
    for gt2 in mrinf_new:
        if gt2 != '':
            fileo.write(gt2)
            fileo.write('\n')
fileo.close()
filei.close()
file2.close()

In [ ]:
##all graphs merged -

In [17]:
outmerged = '../literature_graphs/output_graphs/machine_read_merged_with_closure_v2.0.0.nt'
with open(outmerged,'r') as filei:
    g = filei.read()
mr = g.split('\n')
len(mr)

107057

In [21]:
mr[-2]

'<http://purl.obolibrary.org/obo/CHEBI_47519> <http://purl.obolibrary.org/ob'

In [23]:
outfile = '../literature_graphs/output_graphs/machine_read_merged_with_closure_v2.0.0.nt'
with open(outfile,'w') as fileo:
    for i in range(0, len(mr)-2):
        fileo.write(mr[i])
        fileo.write('\n')
fileo.close()

In [25]:
mr[-2]

'<http://purl.obolibrary.org/obo/CHEBI_47519> <http://purl.obolibrary.org/ob'

#### Merge gpickle files

In [1]:
import networkx as nx
g1 = nx.read_gpickle('../literature_graphs/output_graphs/machineread_semrep_version1.gpickle')
g2 = nx.read_gpickle('../literature_graphs/output_graphs/machineread_reach_version1.gpickle')
ginf = nx.read_gpickle('../literature_graphs/closure_output/machineread_inferred_symmetric_transitive.gpickle')

In [2]:
print(len(g1), len(g2), len(ginf))

4288 6217 2174


In [3]:
def get_stats(g):
    nodes = nx.number_of_nodes(g)
    edges = nx.number_of_edges(g)
    density = nx.density(g)
    avg_deg = float(edges)/nodes
    print(nodes, edges, density, avg_deg)

In [4]:
##compose graphs
g3 = nx.compose(g1,g2)
g_all = nx.compose(g3,ginf)

In [33]:
print('SemRep graph')
get_stats(g1)
print('Reach graph')
get_stats(g2)
print('Inferred graph')
get_stats(ginf)
print('Combined graphs')
get_stats(g_all)

SemRep graph
4288 33594 0.0018274834713764975 7.834421641791045
Reach graph
6217 40245 0.0010414059593728245 6.473379443461477
Inferred graph
2174 18450 0.003905504157192203 8.486660533578657
Combined graphs
8782 84569 0.0010966645002845241 9.629810976998407


In [5]:
##save graph
nx.write_gpickle(g_all, '../literature_graphs/output_graphs/machine_read_merged_v2.0.0.gpickle')

### Merge nodelabels for reach and semrep

In [2]:
import json
with open('ro.json', 'r') as file:
    ro = json.load(file)
len(ro)

1

In [17]:
relation_labels = {}
for node in ro['graphs'][0]['nodes']:
    try:
        if 'lbl' in node.keys():
            relation_labels[node['id']] = node['lbl']
    except:
        print(node)
        break
len(relation_labels)

767

In [21]:
relation_labels['http://purl.obolibrary.org/obo/RO_0002434']

'interacts with'

In [22]:
import pickle

In [23]:
nodelabels_sem = '../literature_graphs/output_graphs/machineread_semrep_NodeLabels.pickle'
nodelabels_r = '../literature_graphs/output_graphs/reach_version1_NodeLabels.pickle'

In [24]:
with open(nodelabels_sem, 'rb') as filep1:
    sem_labels = pickle.load(filep1)
len(sem_labels)

4304

In [25]:
with open(nodelabels_r, 'rb') as filep2:
    r_labels = pickle.load(filep2)
len(r_labels)

6230

In [26]:
##combine labels
all_labels = {**sem_labels, **r_labels}
len(all_labels)

8758

In [27]:
##fix labels of relations using relation_labels
for key in all_labels:
    if key in relation_labels.keys():
        all_labels[key]['label'] = relation_labels[key]
len(all_labels)

8758

In [30]:
all_labels['http://purl.obolibrary.org/obo/RO_0002448']

{'entity_type': 'RELATIONS', 'label': 'directly regulates activity of'}

In [31]:
##save labels
with open('../literature_graphs/output_graphs/machine_read_merged_20230423_NodeLabels.pickle', 'wb') as filep:
    pickle.dump(all_labels, filep)

In [69]:
sem.loc[(sem['subject_obo'] == 'http://purl.obolibrary.org/obo/CHEBI_180536')]['source_text'][139612]

'26692748_ascii.txt.tx.4 However,  the  IC50  values  for the mitragynine and 7-hydroxymitragynine could not be determined  accurately  since  their  inhibitions  on  4-MU glucuronidation were <50%.4-MU glucuronidation was reduced by 7-hydroxymitragynine ranged from 6% +/- 2% to 46% +/- 1% and mitragynine ranged from 14% +/- 4% to 40% +/- 5% at concentrations of 0.01-100 uM [Figure 2].'